<a href="https://colab.research.google.com/github/akshat5002/sales-store-analysis/blob/main/sales_store_analyies_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
# Load Data
data = pd.read_csv("/content/features.csv", parse_dates=["Date"])

# Display basic info
print(data.info())

# Show first few rows
print(data.head())

Business Challenges:

#Identifying High vs. Low-Performing Stores
Challenges:

1.What factors differentiate high-performing stores from struggling ones?
How do markdown strategies, economic conditions, and regional variations impact store performance?
Solutions:

#Factor Analysis:
Analyze key metrics such as sales, markdowns, inventory turnover, and profitability to identify performance drivers.
Use clustering techniques to group stores based on similar characteristics.
Economic and Regional Insights:
Incorporate external factors like CPI (Consumer Price Index), fuel prices, and unemployment rates to understand their impact on store performance.
Identify regions with favorable economic conditions that correlate with high performance.
#Markdown Strategy Evaluation:
Assess the effectiveness of markdowns in driving sales and clearing inventory.
Compare markdown strategies between high and low-performing stores to identify best practices.
# Optimizing Store Strategies Based on Clustering
Challenges:
Should different pricing or inventory strategies be applied to different clusters?
How can markdowns be optimized for better profitability?
Solutions:

#Cluster-Specific Strategies:
For high-performing stores (Cluster 0): Focus on premium pricing, limited markdowns, and high-margin products.
For low-performing stores (Cluster 1): Implement targeted promotions, aggressive markdowns, and inventory optimization to reduce excess stock.
#Markdown Optimization:
Use predictive analytics to determine the optimal timing and depth of markdowns.
Avoid excessive markdowns that erode profitability by balancing inventory turnover with pricing strategies.
#Inventory Management:
Tailor inventory levels to regional demand patterns.
Use clustering insights to forecast demand and reduce overstocking in low-performing regions.
# Data-Driven Decision Making for Growth
Challenges:
Can we group stores with similar performance to create targeted business strategies?
How do external factors (CPI, fuel prices, unemployment) affect store clusters?
Solutions:

#Clustering for Strategy Development:
Group stores into clusters based on performance metrics and external factors.
Develop targeted strategies for each cluster, such as marketing campaigns, pricing adjustments, and operational improvements.
External Factor Analysis:
Use regression or correlation analysis to quantify the impact of external  (eg., CPI, fuel prices, unemployment) on store performance.
Adjust strategies based on regional economic conditions (e.g., reduce prices in areas with high unemployment).
#Continuous Monitoring:
Regularly update clustering models to reflect changes in store performance and external conditions.
Use dashboards and visualizations to track cluster performance and strategy effectiveness.


In [ ]:
print(data.isnull().sum())  # Count missing values in each column


In [ ]:
 #Check missing values
print(data.isnull().sum())

# Fill missing values in numerical columns with median
num_cols = ['Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment']
data[num_cols] = data[num_cols].fillna(data[num_cols].median())

# Fill categorical column
data['IsHoliday'] = data['IsHoliday'].fillna(data['IsHoliday'].mode()[0])

In [ ]:
# Extract date features
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Week'] = data['Date'].dt.isocalendar().week
data['DayOfWeek'] = data['Date'].dt.dayofweek  # Monday=0, Sunday=6


In [ ]:
# Lag features for temperature and fuel price
data['Prev_Week_Temp'] = data.groupby('Store')['Temperature'].shift(7)
data['Prev_Week_Fuel_Price'] = data.groupby('Store')['Fuel_Price'].shift(7)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Correlation Heatmap
plt.figure(figsize=(10,4))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Feature Correlation")
plt.show()

In [ ]:
# Visualize markdown trends over time
plt.figure(figsize=(10,6))
sns.scatterplot(x='Date', y='MarkDown1', data=data, color="green")
plt.title("Markdown 1 Trend Over Time")
plt.show()

In [ ]:
# Compare markdowns on holidays vs non-holidays
sns.boxplot(x='IsHoliday', y='MarkDown1', data=data,color='red')
plt.title("Markdown 1 on Holidays vs Non-Holidays")
plt.show()

ADVANCE STORE LEVEL ANAYLSIS :

#Store Opening Date
 This refers to the specific date when a retail store first opened its doors to customers.

SOLUTIONS : Knowing the store opening date helps in understanding how long the store has been operating, which can be important for analyzing its performance over time.

#Closed Store Identification
 This involves identifying stores that are no longer in operation or have been permanently closed.

SOLUTION : Identifying closed stores is crucial for analyzing overall sales trends and understanding market dynamics, as it helps to assess the impact of closures on sales performance and customer reach.

# Sales Growth/Decline Analysis
 This is the process of examining changes in sales figures over a specific period to determine whether sales are increasing (growth) or decreasing (decline).

SOUTION: Analyzing sales growth or decline helps businesses understand their performance, identify successful strategies, and pinpoint areas that need improvement. It can also inform decisions about inventory, staffing, and marketing efforts.



In [ ]:
# Get the first recorded date for each store
store_opening_dates = data.groupby("Store")["Date"].min().reset_index()
store_opening_dates.columns = ["Store", "Opening_Date"]

print(store_opening_dates.head())

In [ ]:
# Get the last recorded date for each store
store_closing_dates = data.groupby("Store")["Date"].max().reset_index()
store_closing_dates.columns = ["Store", "Last_Active_Date"]

# Define threshold for closing (e.g., stores inactive for 1 year)
latest_date = data["Date"].max()
store_closing_dates["Days_Inactive"] = (latest_date - store_closing_dates["Last_Active_Date"]).dt.days

# Identify stores inactive for more than a year
closed_stores = store_closing_dates[store_closing_dates["Days_Inactive"] > 365]
if(closed_stores.shape[0]==0):
  print('No closed_stores')


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Selecting relevant store-level features for clustering
features = ['Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment']

# Aggregate data at the store level (mean values)
store_data = data.groupby("Store")[features].mean().reset_index()

# Standardize the data
scaler = StandardScaler()
store_data_scaled = scaler.fit_transform(store_data[features])

In [ ]:
# Apply K-Means clustering with 2 clusters
kmeans = KMeans(n_clusters=2, random_state=42)
store_data["Cluster"] = kmeans.fit_predict(store_data_scaled)

# View cluster assignments
print(store_data[["Store", "Cluster"]].head())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,6))
sns.scatterplot(x=store_data["CPI"], y=store_data["Unemployment"], hue=store_data["Cluster"], palette="viridis")
plt.xlabel("CPI (Consumer Price Index)")
plt.ylabel("Unemployment Rate")
plt.title("Store Clustering Based on Economic Indicators")
plt.legend(title="Cluster")
plt.show()

Business Interpretation of Clusters

#Cluster 0 (e.g., High Performance Stores)
Lower unemployment rate, stable fuel prices, and fewer markdowns.
Likely in economically strong regions.

#Cluster 1 (e.g., Low Performance Stores)
Higher markdowns, possibly in areas with economic challenges.
May need targeted promotions or business strategy changes.